In [1]:
%load_ext dockermagic

# Hadoop - multi-node cluster setup 
![Hadoop](https://hadoop.apache.org/elephant.png)

## Create Hadoop base image

### Create docker container

- Ubuntu 18.04 (https://ubuntu.com/)
- Docker (https://www.docker.com/)
    - container based virtualization

In [2]:
%%bash

docker run -d -t --rm --name hadoopimg -h hadoopimg ubuntu:18.04

docker ps

667c063e928b3c2bf6b8451e7314ca81975101fccee6636f6fa9940a5c7b38ba
CONTAINER ID   IMAGE          COMMAND       CREATED         STATUS                  PORTS     NAMES
667c063e928b   ubuntu:18.04   "/bin/bash"   3 seconds ago   Up Less than a second             hadoopimg


### Install Dependencies

- Java 8 (OpenJDK) - https://cwiki.apache.org/confluence/display/HADOOP/Hadoop+Java+Versions
- Other packages: ssh pdsh wget apt-utils

In [3]:
%%dockerexec hadoopimg

# Update package list
apt -qq update > /install.log 2>&1

# Install Hadoop dependencies
apt -qq -f -y install openjdk-8-jdk ssh pdsh >> /install.log 2>&1

# Install other dependencies
apt -qq -f -y install vim wget apt-utils python3 python3-pip \
    ipython3 less unzip sudo net-tools >> /install.log 2>&1

### Install Hadoop

- http://hadoop.apache.org/
- Version 3.2.1
- Base directory: /opt
- User/group: hadoop/hadoop
- Package with binaries (version 3.2.1): https://hadoop.apache.org/releases.html

In [4]:
%%dockerexec hadoopimg

# Enable rwx for all on /opt
chmod 777 /opt

# Create user/group hadoop
useradd -m -U -s /bin/bash hadoop

# Enable sudo for hadoop
sed -i "\$ahadoop  ALL=(ALL) NOPASSWD:ALL" /etc/sudoers

In [5]:
%%bash

# Download package
cd ../pkgs
wget -q -c https://downloads.apache.org/hadoop/common/hadoop-3.2.1/hadoop-3.2.1.tar.gz

# Copy installation package to container
docker cp hadoop-3.2.1.tar.gz hadoopimg:/opt

In [6]:
%%dockerexec -u hadoop hadoopimg

# Modify user/group permissions and unpack file
sudo chown hadoop:hadoop /opt/hadoop-3.2.1.tar.gz
tar -zxf /opt/hadoop-3.2.1.tar.gz -C /opt
rm /opt/hadoop-3.2.1.tar.gz

# Create link
ln -s /opt/hadoop-3.2.1 /opt/hadoop

### Configure environment variables

- Create file /opt/envvars.sh with environment variables

In [7]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/envvars.sh << EOF
export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
export PDSH_RCMD_TYPE=ssh

export HADOOP_HOME=/opt/hadoop
export HADOOP_COMMON_HOME=\${HADOOP_HOME}
export HADOOP_CONF_DIR=\${HADOOP_HOME}/etc/hadoop
export HADOOP_HDFS_HOME=\${HADOOP_HOME}
export HADOOP_MAPRED_HOME=\${HADOOP_HOME}
export HADOOP_YARN_HOME=\${HADOOP_HOME}

export PATH=\${PATH}:\${HADOOP_HOME}/bin:\${HADOOP_HOME}/sbin     

EOF

cat /opt/envvars.sh

export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
export PDSH_RCMD_TYPE=ssh

export HADOOP_HOME=/opt/hadoop
export HADOOP_COMMON_HOME=${HADOOP_HOME}
export HADOOP_CONF_DIR=${HADOOP_HOME}/etc/hadoop
export HADOOP_HDFS_HOME=${HADOOP_HOME}
export HADOOP_MAPRED_HOME=${HADOOP_HOME}
export HADOOP_YARN_HOME=${HADOOP_HOME}

export PATH=${PATH}:${HADOOP_HOME}/bin:${HADOOP_HOME}/sbin     



### Configure passwordless ssh

In [8]:
%%dockerexec -u hadoop hadoopimg

# Disable host key checking
sudo tee -a /etc/ssh/ssh_config << EOF
    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null
EOF

# Create ssh key
ssh-keygen -q -t rsa -P "" -f ~/.ssh/id_rsa

# Copy public key to authorized_keys file
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys

    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null


### Hadoop configuration files

- Hadoop configuration files location: \$HADOOP\_HOME\/etc\/hadoop
- All cluster nodes contain the same files

#### hadoop-env.sh

- Definition of environment variables used by Hadoop processes

In [9]:
%%dockerexec -u hadoop hadoopimg

cat >> /opt/hadoop/etc/hadoop/hadoop-env.sh << EOF
export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
EOF

#### core-site.xml

- Hadoop main configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-project-dist/hadoop-common/core-default.xml

In [10]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/core-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>

<property>
    <name>fs.defaultFS</name>
    <value>hdfs://hadoop:9000</value>
</property>

<property>
    <name>hadoop.proxyuser.hadoop.groups</name>
    <value>*</value>
</property>

<property>
    <name>hadoop.proxyuser.hadoop.hosts</name>
    <value>*</value>
</property>

</configuration>
EOF

#### hdfs-site.xml

- HDFS configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-project-dist/hadoop-hdfs/hdfs-default.xml

In [11]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/hdfs-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>

<property>
    <name>dfs.namenode.name.dir</name>
    <value>/opt/hadoop/data/nameNode</value>
</property>

<property>
    <name>dfs.datanode.data.dir</name>
    <value>/opt/hadoop/data/dataNode</value>
</property>

<property>
    <name>dfs.replication</name>
    <value>2</value>
</property>

<property>
    <name>dfs.blocksize</name>
    <value>33554432</value>
</property>

<property>
    <name>dfs.hosts.exclude</name>
    <value>/opt/hadoop/etc/hadoop/dfs.exclude</value>
</property>

<property>
    <name>dfs.namenode.heartbeat.recheck-interval</name>
    <value>10000</value>
</property>

</configuration>

EOF

#### yarn-site.xml

- YARN configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-yarn/hadoop-yarn-common/yarn-default.xml

In [12]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/yarn-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<configuration>

<property>
    <name>yarn.resourcemanager.hostname</name>
    <value>hadoop</value>
</property>

<property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
</property>

<property>
    <name>yarn.nodemanager.resource.memory-mb</name>
    <value>1536</value>
</property>

<property>
    <name>yarn.scheduler.maximum-allocation-mb</name>
    <value>1536</value>
</property>

<property>
    <name>yarn.scheduler.minimum-allocation-mb</name>
    <value>128</value>
</property>

<property>
    <name>yarn.timeline-service.enabled</name>
    <value>true</value>
</property>

<property>
    <name>yarn.timeline-service.hostname</name>
    <value>hadoop</value>
</property>

<property>
    <name>yarn.system-metrics-publisher.enabled</name>
    <value>true</value>
</property>

<property>
    <name>yarn.log-aggregation-enable</name>
    <value>true</value>
</property>

<property>
    <name>yarn.nm.liveness-monitor.expiry-interval-ms</name>
    <value>10000</value>
</property>

</configuration>
EOF

#### mapred-site.xml

- MapReduce configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-mapreduce-client/hadoop-mapreduce-client-core/mapred-default.xml

In [13]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/mapred-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>

<property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
</property>

<property>
    <name>mapreduce.application.classpath</name>
    <value>/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/mapreduce/lib/*</value>
</property>

<property>
    <name>yarn.app.mapreduce.am.resource.mb</name>
    <value>512</value>
</property>

<property>
    <name>mapreduce.map.memory.mb</name>
    <value>256</value>
</property>

<property>
    <name>mapreduce.reduce.memory.mb</name>
    <value>256</value>
</property>

</configuration>
EOF

#### workers

- List of worker nodes (NodeManager and DataNode)

In [14]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/workers << EOF
hadoop1
hadoop2
hadoop3
EOF

### Commit base image

In [15]:
%%bash

# Create hadoopimg image based on hadoop container
docker commit hadoopimg hadoopimg

# Stop base container
docker stop hadoopimg

sha256:d9fc2c71bd09525c99a4ac728da4bb2faeb6deece36d1a10b03b443edd18601a
hadoopimg


## Create cluster

### Run nodes

In [16]:
%%bash

# MASTER

# Ports
# 9870 - Namenode
# 9868 - Secondary Namenode
# 8088 - ResourceManager
# 19888 - MapReduce Job History
# 8188 - Timeline Service
# 4040 - Spark Application UI
# 8080 - Jupyter

cd ..
docker run -d -t --memory 4g --memory-swap 4g --rm --name hadoop -h hadoop -u hadoop \
    -v "$(pwd)"/pkgs:/opt/pkgs -v "$(pwd)"/notebooks:/opt/notebooks -v "$(pwd)"/datasets:/opt/datasets \
    -p 9870:9870 -p 9868:9868 -p 8088:8088 -p 19888:19888 -p 8188:8188 -p 4040:4040 -p 8080:8080 hadoopimg

# WORKERS

# Ports
# 9864 - DataNode WebUI
# 8042 - NodeManager WebUI

# Hadoop1
docker run -d -t --memory 2g --memory-swap 2g --rm --name hadoop1 -h hadoop1 -u hadoop \
    -p 9864:9864 -p 8042:8042 hadoopimg
# Hadoop2
docker run -d -t --memory 2g --memory-swap 2g --rm --name hadoop2 -h hadoop2 -u hadoop \
    -p 9865:9864 -p 8043:8042  hadoopimg
# Hadoop3
docker run -d -t --memory 2g --memory-swap 2g --rm --name hadoop3 -h hadoop3 -u hadoop \
    -p 9866:9864 -p 8044:8042  hadoopimg

docker ps

08775ab0a1644a065e489ddae1ad9add53ff33b398c2779566269ab1f93fd15b
22c6faf5a700c7ea6a03a2b369f3436213c12aed054271b40b2eac5304515394
222cc5db4d4bb18df3c215d8b2f48fb9cdf607bc2d668c5ab2bb9b499b946f81
e4d4b6209c85bb7569bf1580e4a2a0b62b2dadad8b920d125d467ce2ec156ec7
CONTAINER ID   IMAGE       COMMAND       CREATED         STATUS                  PORTS                                                                                                                                                                      NAMES
e4d4b6209c85   hadoopimg   "/bin/bash"   1 second ago    Up Less than a second   0.0.0.0:8044->8042/tcp, 0.0.0.0:9866->9864/tcp                                                                                                                             hadoop3
222cc5db4d4b   hadoopimg   "/bin/bash"   2 seconds ago   Up 1 second             0.0.0.0:8043->8042/tcp, 0.0.0.0:9865->9864/tcp                                                                                               

### Configure hosts file on all nodes

- /etc/hosts

In [17]:
%%bash

# Get IPs
M=$(docker inspect hadoop | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)
H1=$(docker inspect hadoop1 | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)
H2=$(docker inspect hadoop2 | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)
H3=$(docker inspect hadoop3 | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)

# Create hosts file
cat > hosts << EOF  
$M hadoop
$H1 hadoop1
$H2 hadoop2
$H3 hadoop3
EOF

cat hosts

# Copy to all nodes
docker exec -i -u root hadoop sh -c 'cat >> /etc/hosts' < hosts
docker exec -i -u root hadoop1 sh -c 'cat >> /etc/hosts' < hosts
docker exec -i -u root hadoop2 sh -c 'cat >> /etc/hosts' < hosts
docker exec -i -u root hadoop3 sh -c 'cat >> /etc/hosts' < hosts

# Remove local file
rm hosts

172.17.0.2 hadoop
172.17.0.3 hadoop1
172.17.0.4 hadoop2
172.17.0.5 hadoop3


### Start ssh server on all nodes

In [18]:
%%bash

for HOST in hadoop hadoop1 hadoop2 hadoop3
do
    echo $HOST
    docker exec -u root $HOST service ssh restart
    docker exec -u root $HOST service ssh status
done

hadoop
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running
hadoop1
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running
hadoop2
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running
hadoop3
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running


## Format HDFS on Namenode

In [19]:
%%dockerexec hadoop
source /opt/envvars.sh

hdfs namenode -format -force -nonInteractive

2021-01-28 14:37:38,012 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = hadoop/172.17.0.2
STARTUP_MSG:   args = [-format, -force, -nonInteractive]
STARTUP_MSG:   version = 3.2.1
STARTUP_MSG:   classpath = /opt/hadoop/etc/hadoop:/opt/hadoop/share/hadoop/common/lib/jackson-core-2.9.8.jar:/opt/hadoop/share/hadoop/common/lib/jetty-io-9.3.24.v20180605.jar:/opt/hadoop/share/hadoop/common/lib/audience-annotations-0.5.0.jar:/opt/hadoop/share/hadoop/common/lib/checker-qual-2.5.2.jar:/opt/hadoop/share/hadoop/common/lib/paranamer-2.3.jar:/opt/hadoop/share/hadoop/common/lib/jackson-annotations-2.9.8.jar:/opt/hadoop/share/hadoop/common/lib/animal-sniffer-annotations-1.17.jar:/opt/hadoop/share/hadoop/common/lib/jettison-1.1.jar:/opt/hadoop/share/hadoop/common/lib/jersey-core-1.19.jar:/opt/hadoop/share/hadoop/common/lib/commons-io-2.5.jar:/opt/hadoop/share/hadoop/common/lib/nimbus-jose-jwt-4.41.1.

## Start Hadoop daemons

- manual execution: ```hdfs --daemon start (namenode|datanode)``` and ```yarn --daemon start (resourcemanager|nodemanager)```
- auxilliary scripts to run all processes on the cluster: start-dfs.sh (HDFS) and start-yarn.sh (YARN)
- some services still need to be executed manually (timelineserver, historyserver)

In [20]:
%%dockerexec hadoop
source /opt/envvars.sh

# HDFS
start-dfs.sh

# YARN
start-yarn.sh

# timelineserver
yarn --daemon start timelineserver

# historyserver
mapred --daemon start historyserver

Starting namenodes on [hadoop]
hadoop: Warning: Permanently added 'hadoop,172.17.0.2' (ECDSA) to the list of known hosts.
Starting datanodes
hadoop2: Warning: Permanently added 'hadoop2,172.17.0.4' (ECDSA) to the list of known hosts.
hadoop1: Warning: Permanently added 'hadoop1,172.17.0.3' (ECDSA) to the list of known hosts.
hadoop3: Warning: Permanently added 'hadoop3,172.17.0.5' (ECDSA) to the list of known hosts.
hadoop2: WARNING: /opt/hadoop-3.2.1/logs does not exist. Creating.
hadoop3: WARNING: /opt/hadoop-3.2.1/logs does not exist. Creating.
hadoop1: WARNING: /opt/hadoop-3.2.1/logs does not exist. Creating.
Starting secondary namenodes [hadoop]
hadoop: Warning: Permanently added 'hadoop,172.17.0.2' (ECDSA) to the list of known hosts.
Starting resourcemanager
Starting nodemanagers
hadoop1: Warning: Permanently added 'hadoop1,172.17.0.3' (ECDSA) to the list of known hosts.
hadoop2: Warning: Permanently added 'hadoop2,172.17.0.4' (ECDSA) to the list of known hosts.
hadoop3: Warning:

In [21]:
%%bash

# Listing all processes
for HOST in hadoop hadoop1 hadoop2 hadoop3; do
    echo $HOST
    docker exec $HOST jps
done

hadoop
498 SecondaryNameNode
291 NameNode
916 ApplicationHistoryServer
1158 Jps
972 JobHistoryServer
765 ResourceManager
hadoop1
106 DataNode
222 NodeManager
318 Jps
hadoop2
224 NodeManager
321 Jps
108 DataNode
hadoop3
225 NodeManager
322 Jps
109 DataNode


## Create HDFS directories

In [22]:
%%dockerexec hadoop
source /opt/envvars.sh

hdfs dfs -mkdir -p /user/hadoop
hdfs dfs -chown hadoop:hadoop /user/hadoop
hdfs dfs -mkdir /tmp
hdfs dfs -chmod 777 /tmp

mkdir: `/tmp': File exists


## Install and run Jupyter on master node

In [23]:
%%dockerexec hadoop

# pip3 -q install notebook
pip3 -q install jupyterlab

IP=$(ifconfig eth0 | grep inet | awk '{ print $2 }')

cd /opt

export SHELL=/bin/bash
# nohup /home/hadoop/.local/bin/jupyter-notebook --ip=$IP --port=8080 --NotebookApp.token='' --NotebookApp.password='' --NotebookApp.notebook_dir='/' --no-browser &
nohup /home/hadoop/.local/bin/jupyter-lab --ip=$IP --port=8080 --notebook-dir='/' --ServerApp.token='' --ServerApp.password='' --no-browser &

echo $! > /tmp/jupyter.pid

# To kill
# kill $(cat /tmp/jupyter.pid)

[I 2021-01-28 14:45:31.653 ServerApp] jupyterlab | extension was successfully linked.
[I 2021-01-28 14:45:31.661 ServerApp] Writing notebook server cookie secret to /home/hadoop/.local/share/jupyter/runtime/jupyter_cookie_secret
[W 2021-01-28 14:45:31.694 ServerApp] All authentication is disabled.  Anyone who can connect to this server will be able to run code.
[I 2021-01-28 14:45:31.709 LabApp] JupyterLab extension loaded from /home/hadoop/.local/lib/python3.6/site-packages/jupyterlab
[I 2021-01-28 14:45:31.709 LabApp] JupyterLab application directory is /home/hadoop/.local/share/jupyter/lab
[I 2021-01-28 14:45:31.719 ServerApp] jupyterlab | extension was successfully loaded.
[I 2021-01-28 14:45:31.728 ServerApp] Serving notebooks from local directory: /
[I 2021-01-28 14:45:31.728 ServerApp] Jupyter Server 1.2.2 is running at:
[I 2021-01-28 14:45:31.728 ServerApp] http://172.17.0.2:8080/lab
[I 2021-01-28 14:45:31.728 ServerApp]  or http://127.0.0.1:8080/lab
[I 2021-01-28 14:45:31.728 

## Access web interfaces

- Jupyterlab
    - http://localhost:8080
- Master - hadoop
    - Resource Manager: http://localhost:8088
    - NameNode: http://localhost:9870
    - Secondary NameNode: http://localhost:9868
    - MapReduce Job History: http://localhost:19888
    - Timeline Service: http://localhost:8188
- Workers
    - hadoop1
        - NodeManager: http://localhost:8042
        - DataNode: http://localhost:9864
    - hadoop2
        - NodeManager: http://localhost:8043
        - DataNode: http://localhost:9865
    - hadoop3
        - NodeManager: http://localhost:8044
        - DataNode: http://localhost:9866

## Run mapreduce Pi example

In [24]:
%%dockerexec hadoop
source /opt/envvars.sh
cd /opt/hadoop/share/hadoop/mapreduce

hadoop jar ./hadoop-mapreduce-examples-3.2.1.jar pi 6 10000

Number of Maps  = 6
Samples per Map = 10000
2021-01-28 14:47:37,074 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
Wrote input for Map #0
2021-01-28 14:47:38,659 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
Wrote input for Map #1
2021-01-28 14:47:38,828 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
Wrote input for Map #2
2021-01-28 14:47:39,306 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
Wrote input for Map #3
2021-01-28 14:47:41,029 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
Wrote input for Map #4
2021-01-28 14:47:41,216 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
Wrote i

# SHUTDOWN PROCEDURE

## Stop Jupyter

In [ ]:
%%dockerexec hadoop

kill $(cat /tmp/jupyter.pid)

## Stop Hadoop daemons

In [ ]:
%%dockerexec hadoop
source /opt/envvars.sh

stop-dfs.sh
stop-yarn.sh
yarn --daemon stop timelineserver
mapred --daemon stop historyserver

## Stop Docker containers

In [ ]:
%%bash

for HOST in hadoop hadoop1 hadoop2 hadoop3; do
    docker stop $HOST
done

docker ps